# GPU를 사용해 학습 가속하기

이번 튜토리얼에서, 다음을 배우게 됩니다.

* 그래프와 피처 데이터를 GPU로 복사하는 방법
* GNN 모델을 GPU 위에 학습하는 방법


In [1]:
# Check cuda version
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
pip install dgl-cu110

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 2.8 MB/s eta 0:00:00


In [3]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


Using backend: pytorch


## 그래프와 피처 데이터를 GPU에 복사

먼저 이전 세션에 사용한 Zachery의 카라테 클럽 그래프와 노드 라벨를 로드합니다.

In [4]:
import pandas as pd

def load_zachery():
    nodes_data = pd.read_csv('https://github.com/myeonghak/DGL-tutorial/raw/master/data/nodes.csv')
    edges_data = pd.read_csv('https://github.com/myeonghak/DGL-tutorial/raw/master/data/edges.csv')
    src = edges_data['Src'].to_numpy()
    dst = edges_data['Dst'].to_numpy()
    g = dgl.graph((src, dst))
    club = nodes_data['Club'].to_list()
    # Convert to categorical integer values with 0 for 'Mr. Hi', 1 for 'Officer'.
    club = torch.tensor([c == 'Officer' for c in club]).long()
    # We can also convert it to one-hot encoding.
    club_onehot = F.one_hot(club)
    g.ndata.update({'club' : club, 'club_onehot' : club_onehot})
    return g

In [5]:
#from tutorial_utils import load_zachery

# ----------- 0. load graph -------------- #
g = load_zachery()
print(g)

Graph(num_nodes=34, num_edges=156,
      ndata_schemes={'club': Scheme(shape=(), dtype=torch.int64), 'club_onehot': Scheme(shape=(2,), dtype=torch.int64)}
      edata_schemes={})


이제 그래프와 모든 그래프의 피처 데이터는 CPU에 적재되어 있습니다. `to` API를 사용해 다른 연산장치로 복사해 보세요.

In [6]:
print('Current device:', g.device)
g = g.to('cuda:0')
print('New device:', g.device)

Current device: cpu
New device: cuda:0


Verify that features are also copied to GPU.

In [7]:
print(g.ndata['club'].device)
print(g.ndata['club_onehot'].device)

cuda:0
cuda:0


## GNN 모델을 GPU에 생성하기

이 스텝은 CNN이나 RNN 모델을 GPU에 생성하는 것과 같습니다.  
PyTorch에서, `to` API를 사용해 이를 수행할 수 있습니다.

In [8]:
# ----------- 1. node features -------------- #
node_embed = nn.Embedding(g.number_of_nodes(), 5)  # 각 노드는 5차원의 임베딩을 가지고 있습니다.
# Copy node embeddings to GPU
node_embed = node_embed.to('cuda:0')
inputs = node_embed.weight                         # 노드 피처로써 이 임베딩 가중치를 사용합니다.
nn.init.xavier_uniform_(inputs)

Parameter containing:
tensor([[ 0.0512,  0.1105, -0.3783, -0.3138, -0.0261],
        [-0.3697, -0.0519, -0.1647, -0.1861,  0.1402],
        [ 0.2388, -0.2833, -0.2277, -0.2604,  0.1609],
        [-0.0015, -0.1800, -0.0628, -0.3103, -0.2310],
        [ 0.3619,  0.2736, -0.0693, -0.3774, -0.3645],
        [ 0.0130, -0.3617, -0.0439,  0.0451, -0.3525],
        [-0.3158, -0.1318, -0.2977,  0.3522, -0.3351],
        [-0.2375, -0.1198, -0.0169,  0.2250,  0.3449],
        [-0.0928, -0.2827, -0.2498, -0.2203,  0.1367],
        [-0.1820,  0.0456, -0.2142,  0.2698, -0.0167],
        [ 0.2905,  0.2195,  0.1550,  0.0767, -0.0195],
        [-0.0318, -0.2610,  0.2051,  0.3398,  0.2931],
        [-0.1404,  0.0501,  0.0910,  0.1598, -0.3840],
        [-0.0805,  0.3095,  0.2779, -0.2318,  0.2361],
        [ 0.2033, -0.3618, -0.0139, -0.3582,  0.1457],
        [ 0.2887,  0.1012,  0.1058, -0.1653, -0.3283],
        [ 0.3444,  0.1393, -0.2606,  0.0716,  0.3879],
        [ 0.2936,  0.2358, -0.0281,  0.2174

커뮤니티의 라벨은 `'club'`이라는 노드 피처에 저장되어 있습니다. (0은 instructor의 커뮤니티, 1은 club president의 커뮤니티).  
오로지 0과 33번 노드에만 라벨링 되어 있습니다.

In [9]:
labels = g.ndata['club']
labeled_nodes = [0, 33]
print('Labels', labels[labeled_nodes])

Labels tensor([0, 1], device='cuda:0')


## GraphSAGE 모델 정의하기

우리의 모델은 2개 레이어로 구성되어 있는데, 각각 새로운 노드 표현(representation)을 이웃의 정보를 통합함으로써 계산합니다.  
수식은 다음과 같습니다.  


$$
h_{\mathcal{N}(v)}^k\leftarrow \text{AGGREGATE}_k\{h_u^{k-1},\forall u\in\mathcal{N}(v)\}
$$

$$
h_v^k\leftarrow \sigma\left(W^k\cdot \text{CONCAT}(h_v^{k-1}, h_{\mathcal{N}(v)}^k) \right)
$$

DGL은 많은 유명한 이웃 통합(neighbor aggregation) 모듈의 구현체를 제공합니다. 모두 쉽게 한 줄의 코드로 호출하여 사용할 수 있습니다.  
지원되는 모델의 전체 리스트는 [graph convolution modules](https://docs.dgl.ai/api/python/nn.pytorch.html#module-dgl.nn.pytorch.conv)에서 보실 수 있습니다.

In [10]:
from dgl.nn import SAGEConv

# ----------- 2. create model -------------- #
# 2개의 레이어를 가진 GraphSAGE 모델 구축
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, num_classes, 'mean')
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h
    
# 주어진 차원의 모델 생성
# 인풋 레이어 차원: 5, 노드 임베딩
# 히든 레이어 차원: 16
# 아웃풋 레이어 차원: 2, 클래스가 2개 있기 때문, 0과 1

net = GraphSAGE(5, 16, 2)

네트워크를 GPU에 복사함

In [11]:
net = net.to('cuda:0')

In [12]:
# ----------- 3. set up loss and optimizer -------------- #
# 이 경우, 학습 루프의 손실

optimizer = torch.optim.Adam(itertools.chain(net.parameters(), node_embed.parameters()), lr=0.01)

# ----------- 4. training -------------------------------- #
all_logits = []
for e in range(100):
    # forward
    logits = net(g, inputs)
    
    # 손실 계산
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[labeled_nodes], labels[labeled_nodes])
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    all_logits.append(logits.detach())
    
    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

In epoch 0, loss: 0.7012346386909485
In epoch 5, loss: 0.3282551169395447
In epoch 10, loss: 0.1220165491104126
In epoch 15, loss: 0.03706631809473038
In epoch 20, loss: 0.01077364757657051
In epoch 25, loss: 0.0036164354532957077
In epoch 30, loss: 0.001538184704259038
In epoch 35, loss: 0.0008244633208960295
In epoch 40, loss: 0.0005310544511303306
In epoch 45, loss: 0.0003911085077561438
In epoch 50, loss: 0.0003160333726555109
In epoch 55, loss: 0.00027152185793966055
In epoch 60, loss: 0.0002429189917165786
In epoch 65, loss: 0.00022313484805636108
In epoch 70, loss: 0.00020829649292863905
In epoch 75, loss: 0.00019649715977720916
In epoch 80, loss: 0.00018648550030775368
In epoch 85, loss: 0.00017766558448784053
In epoch 90, loss: 0.00016956074978224933
In epoch 95, loss: 0.00016193260671570897


In [13]:
# ----------- 5. check results ------------------------ #
pred = torch.argmax(logits, axis=1)
print('Accuracy', (pred == labels).sum().item() / len(pred))

Accuracy 0.8529411764705882


**한 GPU 메모리에 그래프와 피처 데이터를 적재할 수 없으면 어떻게 하나요?** 

* GNN을 천제 그래프에 대해 수행하는 대신에, 몇몇 subgraph에 대해 수행해 수렴시켜보세요.
* 다른 샘플을 다른 GPU에 올림으로써 더 빠른 가속을 경험해 보세요.
* 그래프를 여러 머신에 분할하여 분산된 형태로 학습시켜보세요.

추후에 이러한 방법론을 각각 살펴볼 예정입니다.